In [1]:
import sys
base_directory = "/Users/johnbarrera/Documents/Projects/world_bank/Climate-and-Disaster-Risk-Management-for-Health-Systems/"
sys.path.append(f"{base_directory}")

In [2]:
from src.utils.config_reader import ConfigReader, Logger
from src.utils.utils import GeoDataFrameOperations
from src.utils.file_pocessor import FileLister, FileProcessor

In [3]:
# Create an instance of the Logger
log_directory = "/Users/johnbarrera/Documents/Projects/world_bank/Climate-and-Disaster-Risk-Management-for-Health-Systems/data/nepal/outputs/log"
log_file_name = "processing"
logger = Logger(log_directory, log_file_name)

In [4]:
# Readding the configuration file to the preprocessing
config_file_path = "config/nepal/data_setup.yaml"
config_file_path = f"{base_directory}{config_file_path}"

try:
    config_data = ConfigReader.read_yaml_file(config_file_path)
    # config_data = config_data['processing']
    txt_msg = "Content of {} file successfully read".format(config_file_path)
    logger.info(txt_msg)
except Exception as e:
    txt_msg = f"Error reading configuration file: {str(e)}"
    logger.error(txt_msg)

INFO: Content of /Users/johnbarrera/Documents/Projects/world_bank/Climate-and-Disaster-Risk-Management-for-Health-Systems/config/nepal/data_setup.yaml file successfully read


In [5]:
# Administrative region
path_map = f"{base_directory}{config_data['adminitrative_maps']}"
data_map = FileProcessor.read_geopackage(path_map)

# Hazards list files
path_hazards = f"{base_directory}{config_data['preprocessed_hazards']}"
data_hazards = FileLister.list_files(path_hazards)

In [6]:
import pandas as pd
import geopandas as gpd

In [96]:
def compute_exposure_in_points(object_file, hazard_files):
    data_object = FileProcessor.read_geopackage(object_file)
    data_object = data_object.reset_index(drop=False)
    data_object = data_object.rename(columns={'index': 'temp_index'})
    data_object_temp = data_object[['temp_index','geometry']]
    data_exposure = data_object_temp.copy()
    for file in hazard_files:
        hazard_name = file[0]
        print(hazard_name)
        data_hazard = FileProcessor.read_geopackage(file[1])
        data_hazard = data_hazard.rename(columns={'damage': hazard_name})
        data_hazard = gpd.sjoin(data_object_temp, data_hazard, how='left', predicate='intersects')
        data_hazard[hazard_name] = data_hazard[hazard_name].fillna('no damage')
        data_hazard = data_hazard[['temp_index', hazard_name]]
        data_exposure = gpd.GeoDataFrame.merge(data_exposure, data_hazard, on='temp_index', suffixes=('', '_gdf2'))
        del data_hazard
    del data_object_temp
    del data_exposure['geometry']
    data_exposure = gpd.GeoDataFrame.merge(data_object, data_exposure, on='temp_index', suffixes=('', '_gdf2'))
    del data_exposure['temp_index']
    return data_exposure

In [97]:
# Task_list
path_taks_results = f"{base_directory}{config_data['processing']['output_file']}"
# print(path_taks_results)
# resultado = None
# resultado2 = None
for task in config_data['processing']['tasks']:
    task_name = task['name']
    task_type = task['type']
    
    if task_type == "exposure_population":
        path_populations = f"{base_directory}{task['population_sources']}"
        data_populations = FileLister.list_files(path_populations)
        for object_file in data_populations:
            object_name = object_file[0]
            object_file = object_file[1]
            print(f'Prosessing: {object_name}')
            data_exposure = compute_exposure_in_points(object_file, data_hazards)
            columns_to_keep = [x for x in data_exposure.columns if x not in data_map.columns] + ['geometry']
            data_exposure = gpd.sjoin(data_exposure[columns_to_keep], data_map, how='left', predicate='intersects')
            del data_exposure['index_right']
            output_name = f'{task_type}_{object_name}.gpkg'
            FileProcessor.save_to_geopackage(data_exposure, path_taks_results, output_name)
        pass
    
    if task_type == "exposure_infrastructure":
        path_infrastructures = f"{base_directory}{task['infrastructures_sources']}"
        data_infrastructures = FileLister.list_files(path_infrastructures)
        resultado = data_infrastructures
        for object_file in data_infrastructures:
            object_name = object_file[0]
            object_file = object_file[1]
            print(f'Prosessing: {object_name}')
            data_exposure = compute_exposure_in_points(object_file, data_hazards)
            columns_to_keep = [x for x in data_exposure.columns if x not in data_map.columns] + ['geometry']
            data_exposure = gpd.sjoin(data_exposure[columns_to_keep], data_map, how='left', predicate='intersects')
            del data_exposure['index_right']
            output_name = f'{task_type}_{object_name}.gpkg'
            FileProcessor.save_to_geopackage(data_exposure, path_taks_results, output_name)
    
    if task_type == "risk_index":
        pass
    

Prosessing: population_4326
earthquakes_period_475
landslides_historical
earthquakes_period_1500
earthquakes_period_2475
earthquakes_period_975
Prosessing: healt_facilities_infrastructure
earthquakes_period_475
landslides_historical
earthquakes_period_1500
earthquakes_period_2475
earthquakes_period_975
Prosessing: PHC_infrastructure
earthquakes_period_475
landslides_historical
earthquakes_period_1500
earthquakes_period_2475
earthquakes_period_975


In [94]:
data_exposure.head()
data_exposure.columns

Index(['GHFD_ID', 'HF_ID_N', 'HF_N_RO', 'HF_N_LOC', 'HF_T_RO', 'HF_T_LO',
       'HF_OWN', 'HF_ADD_STR', 'HF_ADD_NO', 'HF_ADD_PC', 'HF_ADD_CN', 'LAT',
       'LONG', 'S_COOR', 'M_COOR', 'AC_COOR', 'earthquakes_period_475',
       'geometry', 'index_right', 'ADM1_C', 'ADM1_N_RO', 'ADM2_C', 'ADM2_N_RO',
       'ADM3_C', 'ADM3_N_RO'],
      dtype='object')

In [17]:
# Task_list
path_taks_results = f"{base_directory}{config_data['processing']['output_file']}"
files_results = FileLister.list_files(path_taks_results)

for file_result in files_results[:2]:
    file_name = file_result[0]
    file_result = file_result[1]
    print(f'Prosessing {file_name} ...')
    data_result = FileProcessor.read_geopackage(file_result)

Prosessing exposure_infrastructure_healt_facilities_infrastructure ...
Prosessing exposure_infrastructure_PHC_infrastructure ...


In [18]:
data_result.head()

,GHFD_ID,HF_ID_N,HF_N_RO,HF_N_LOC,HF_T_RO,HF_T_LO,HF_OWN,HF_ADD_STR,HF_ADD_NO,HF_ADD_PC,...,earthquakes_period_2475,earthquakes_period_975,index_right,ADM1_C_right,ADM1_N_RO_right,ADM2_C_right,ADM2_N_RO_right,ADM3_C_right,ADM3_N_RO_right,geometry
0,None,None,Aadhaebhut Swastha Kendra,None,Primary Health Care Center,None,None,None,None,None,...,substantial damage,substantial damage,155,NPL-ADM1-38925275B31954132,Province 2,NPL-ADM2-48590121B79767844,SAPTARI,NPL-ADM3-92635248B74966006,Tilathi Koiladi,POINT (86.81394 26.49696)
1,None,None,Aadharbhut Swastha Sewa Kendra,None,Primary Health Care Center,None,None,None,None,None,...,substantial damage,substantial damage,155,NPL-ADM1-38925275B31954132,Province 2,NPL-ADM2-48590121B79767844,SAPTARI,NPL-ADM3-92635248B74966006,Tilathi Koiladi,POINT (86.82662 26.51304)
2,None,None,Aadharbhut Swasthya Sewa Kendra Chhadekholaa,आधारभूत स्वास्थ्य सेवा केन्द्र छदेखोला,Primary Health Care Center,None,Local Government,None,None,None,...,substantial damage,substantial damage,617,NPL-ADM1-38925275B17766335,Karnali,NPL-ADM2-48590121B24358344,DAILEKH,NPL-ADM3-92635248B3555706,Chamunda Bindrasaini,POINT (81.54452 28.93257)
3,None,None,Aadharubhut Swaasthya Sewa Kendra,आधारभूत स्वास्थ्य सेव केन्द्र,Primary Health Care Center,None,Local Government,None,None,None,...,substantial damage,substantial damage,617,NPL-ADM1-38925275B17766335,Karnali,NPL-ADM2-48590121B24358344,DAILEKH,NPL-ADM3-92635248B3555706,Chamunda Bindrasaini,POINT (81.50951 28.90983)
4,None,None,Aahal Dada Urban Health Center UHC,None,Primary Health Care Center,None,None,None,None,None,...,substantial damage,substantial damage,327,NPL-ADM1-38925275B3223679,Bagmati,NPL-ADM2-48590121B615949,DHADING,NPL-ADM3-92635248B55308155,Nilakantha,POINT (84.97833 27.90778)


In [ ]:
from src.utils.config_reader import ConfigReader

config_reader = ConfigReader()
config_file_path = "config/nepal/setup_preprocessing.yaml"
config_data = config_reader.read_configuration_file(f"{base_directory}{config_file_path}")
config_data = config_data['preprocessing']

# population_exposure_config = None
# for task in config_data:
#     print(task)
    
config_data

In [ ]:
import os
import sys

In [ ]:
import pandas as pd
import numpy as np
import shapefile as shp
import osmnx as ox
import contextily as ctx
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.express as px
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D  # for legend handle
from matplotlib_scalebar.scalebar import ScaleBar
from sklearn.metrics.pairwise import haversine_distances
from shapely.geometry import Point
from pyproj import Proj, transform
import math
from matplotlib.colors import Normalize

# sys.path.append(os.path.join(os.path.dirname(os.getcwd()), "../src"))
# import functions_support as fsupport

import importlib

# importlib.reload(fsupport)

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

import hashlib
from datetime import datetime, timedelta

In [ ]:
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), "/Users/johnbarrera/Documents/Projects/world_bank/Climate-and-Disaster-Risk-Management-for-Health-Systems"))


In [ ]:
import rasterio
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
import geopandas as gpd
from tqdm import tqdm

In [ ]:
input_path = "../data/nepal/inputs/pga_475y.tif"
input_path2 = "../data/nepal/inputs/pga_specifications.xlsx"
output_path = "../data/nepal/outputs/map/Peak_Ground_Acceleration.gpkg"


In [ ]:
# Abre el archivo TIF
with rasterio.open(input_path) as src:
    # Imprime información sobre el archivo
    print(src.profile)

    # Lee todas las bandas y guarda los valores en una matriz
    data = src.read()

    # Imprime el número de bandas y el tamaño de la matriz
    print(f'Número de bandas: {src.count}')
    print(f'Tamaño de la matriz: {data.shape}')



In [ ]:
# Definir una función para transformar las coordenadas de los píxeles a coordenadas de mapa
def pixel_to_map_coordinates(transform, col, row):
    x, y = transform * (col, row)
    return x, y

# Abrir el archivo TIF
with rasterio.open(input_path) as src:
    # Leer todas las bandas y guardar los valores en una matriz
    data = src.read()

    # Obtener la transformación de coordenadas de píxeles a coordenadas de mapa
    transform = src.transform

    # Crear una lista vacía para almacenar los datos de cada polígono
    polygons = []

    # Iterar sobre cada banda y crear polígonos para cada píxel con valor distinto de cero
    for i in range(src.count):
        band_data = data[i, :, :]

        for row in tqdm(range(band_data.shape[0])):
            for col in range(band_data.shape[1]):
                # Obtener el valor del píxel
                value = band_data[row, col]

                # Si el valor es cero, ignorar el píxel
                if value > 0:
                    #continue

                    # Calcular las coordenadas de los cuatro vértices del polígono
                    x1, y1 = pixel_to_map_coordinates(transform, col, row)
                    x2, y2 = pixel_to_map_coordinates(transform, col + 1, row)
                    x3, y3 = pixel_to_map_coordinates(transform, col + 1, row + 1)
                    x4, y4 = pixel_to_map_coordinates(transform, col, row + 1)

                    # Crear el polígono a partir de los vértices
                    poly = Polygon([(x1, y1), (x2, y2), (x3, y3), (x4, y4)])

                    # Agregar el polígono y su información a la lista de polígonos
                    polygon_data = {
                        'band': i+1,
                        'value': value,
                        'geometry': poly
                    }
                    polygons.append(polygon_data)
                    
# Crear un GeoDataFrame a partir de la lista de polígonos
gdf = gpd.GeoDataFrame(polygons)


In [ ]:
# Plot the GeoDataFrame with colors based on the 'value' column
gdf.plot(column='value', cmap='viridis', legend=True, figsize=(10, 10))

# Add title and labels
plt.title('Map with Colors per Value Column')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Show the plot
plt.show()

In [ ]:

pga_specifications = pd.read_excel(f"{input_path2}")  

# dtype_mapping = {
#     'Acceleration_min': float,
#     'Acceleration_max': float, 
# }

# pga_specifications = pd.read_csv(f"{input_path2}", dtype=dtype_mapping)
pga_specifications

In [ ]:
gdf

In [ ]:
def identify_interval(value, df, min_col, max_col):
    for index, row in df.iterrows():
        min_value = row[min_col]
        max_value = row[max_col]
        if min_value <= value and value < max_value:
            return row['Instrumental_Intensity'], row['Acceleration_g'], row['Velocity_cmxs'], row['Perceived_shaking'], row['Potential_damage']
        

def gdf_interval_df(gdf, df, value_col, min_col, max_col):
    '''
    '''
    gdf2 = gdf.copy()
    resultado = gdf2.apply(lambda x: identify_interval(x[value_col], df, min_col, max_col), axis=1)
    gdf2['Instrumental_Intensity'] = [x[0] for x in resultado]
    gdf2['Acceleration_g'] = [x[1] for x in resultado]
    gdf2['Velocity_cmxs'] = [x[2] for x in resultado]
    gdf2['Perceived_shaking'] = [x[3] for x in resultado]
    gdf2['Potential_damage'] = [x[4] for x in resultado]

    return gdf2

gdf2 = gdf_interval_df(gdf, pga_specifications, value_col='value', min_col='Acceleration_min', max_col='Acceleration_max')
gdf2

In [ ]:
gdf2.to_file(output_path, driver="GPKG")

In [ ]:
set(gdf2['Instrumental_Intensity'])